# Random Forest

In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV

In [11]:
df = pd.read_csv("../df_postprocessing_unscaled.csv")
df_2021 = pd.read_csv("../df_2021_postprocessing_unscaled.csv")
df.head()

,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,DRB_per,AST_per,TO_per,...,team_William & Mary,team_Winston Salem St.,team_Winthrop,team_Wisconsin,team_Wofford,team_Wright St.,team_Wyoming,team_Xavier,team_Yale,team_Youngstown St.
0,32,64.0,108.3,18.1,53.0,53.33,2.3,5.9,14.6,14.1,...,0,0,0,0,0,0,0,0,0,0
1,11,1.0,121.1,16.8,75.0,75.00,0.0,0.0,15.8,24.8,...,0,0,0,0,0,0,0,0,0,0
2,27,81.8,99.8,27.7,49.9,53.76,6.9,17.6,15.6,18.6,...,0,0,0,0,0,0,0,0,0,0
3,35,76.2,118.1,16.4,58.8,60.93,4.7,11.5,12.3,15.1,...,0,0,0,0,0,0,0,0,0,0
4,29,9.5,60.7,6.9,8.3,14.93,3.2,13.9,7.7,6.1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df.columns[:80]

Index(['GP', 'Min_per', 'Ortg', 'usg', 'eFG', 'TS_per', 'ORB_per', 'DRB_per',
       'AST_per', 'TO_per', 'FTM', 'FTA', 'FT_per', 'twoPM', 'twoPA',
       'twoP_per', 'TPM', 'TPA', 'TP_per', 'blk_per', 'stl_per', 'ftr', 'yr',
       'num', 'porpag', 'adjoe', 'pfr', 'pid', 'ast/tov', 'rimmade',
       'rimmade+rimmiss', 'midmade', 'midmade+midmiss',
       'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+midmiss)', 'dunksmade',
       'dunksmiss+dunksmade', 'pick', 'drtg', 'adrtg', 'dporpag', 'stops',
       'bpm', 'obpm', 'dbpm', 'gbpm', 'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb',
       'treb', 'ast', 'stl', 'blk', 'pts', 'round', 'round_1_pick',
       'overall_pick', 'first_round_drafted', 'second_round_drafted',
       'role_Combo G', 'role_PF/C', 'role_Pure PG', 'role_Scoring PG',
       'role_Stretch 4', 'role_Wing F', 'role_Wing G', 'ht_month_Jul',
       'ht_month_Jun', 'ht_month_May', 'ht_day_1', 'ht_day_10', 'ht_day_11',
       'ht_day_2', 'ht_day_3', 'ht_day_4', 'ht_day_5', 'ht_da

In [13]:
df[['round', 'round_1_pick',
       'overall_pick', 'first_round_drafted', 'second_round_drafted']].describe()

,round,round_1_pick,overall_pick,first_round_drafted,second_round_drafted
count,49756.000000,49756.000000,49756.000000,49756.000000,49756.000000
mean,0.014631,0.152404,0.292889,0.005266,0.004683
std,0.154219,1.745415,3.388681,0.072375,0.068272
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,30.000000,60.000000,1.000000,1.000000


In [14]:
X_train = df.drop(['round', 'round_1_pick', 'overall_pick', 
            'first_round_drafted', 'second_round_drafted'], axis=1)
y_train = df['first_round_drafted']

In [15]:
X_test = df_2021.drop(['round', 'round_1_pick', 'overall_pick', 
            'first_round_drafted', 'second_round_drafted'], axis=1)
y_test = df_2021['first_round_drafted']

In [ ]:
df_2021 'second_round_drafted']

In [16]:
# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

CV_rf = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=5, scoring='roc_auc')
CV_rf.fit(X_train, y_train)
best_rf = CV_rf.best_estimator_

/Users/tommasosalvatori/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/tommasosalvatori/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/tommasosalvatori/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/Users/tommasosalvatori/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 638, in _validate_params
    

In [35]:
# Evaluate the model
pred = best_rf.predict_proba(X_test)[:,1]
print("Classification Report:")
print("Confusion Matrix:")
#print(f"ROC-AUC Score: {roc_auc_score(y_test, predictions)}")

Classification Report:
Confusion Matrix:


In [36]:
pred

array([0.00000000e+00, 0.00000000e+00, 5.27704485e-05, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [42]:
pred_df = pd.DataFrame(pred)
pred_df.columns = ['pred']

# df_2021 = df_2021.reset_index()

merged_df = df_2021.merge(pred_df, left_index=True, right_index=True)

selected_by_model = merged_df.sort_values('pred', ascending=False).head(46)
selected_by_model.second_round_drafted.value_counts()

second_round_drafted
0    33
1    13
Name: count, dtype: int64

In [30]:
merged_df.sort_values('pred', ascending=False)[['pred', 'first_round_drafted']]

,pred,first_round_drafted
3745,1,1
0,0,0
3092,0,0
3091,0,0
3090,0,0
...,...,...
1543,0,0
1542,0,0
1541,0,0
1540,0,0
